In [9]:
import yfinance as yf
import pandas as pd
import numpy as np


ETFS = [{'ticker': 'IWMO.MI', 'name': 'MSCI World Momentum', 'isin': 'IE00BP3QZ825', 'ter': 0.3, 'value': 35},
         {'ticker': 'MVOL.MI', 'name': 'MSCI Minimum Volatility', 'isin': 'IE00B8FHGS14', 'ter': 0.3, 'value': 25},
         {'ticker': 'SGLD.MI', 'name': 'Invesco Gold', 'isin': 'IE00B579F325', 'ter': 0.12, 'value': 10},
         {'ticker': 'SXRQ.DE', 'name': 'GGOV Bond Eur', 'isin': 'IE00B3VTN290', 'ter': 0.2, 'value': 20},
         {'ticker': 'LEONIA.MI', 'name': 'Amundi Overnight return', 'isin': 'FR0010510800', 'ter': 0.1, 'value': 10}]


In [10]:
min_date = '2022-04-01'
for etf in ETFS:
    ticker = etf['ticker']
    name = etf['name']
    
    fondo = yf.Ticker(ticker)
    dati = fondo.history(period="10y", interval="1mo")
    # print the minimum date
    print(f"Minimum date for {name} ({ticker}): {dati.index.min()}")
    # add the data to the ETFS array
    etf['data'] = dati[dati.index >= min_date]


Minimum date for MSCI World Momentum (IWMO.MI): 2015-08-01 00:00:00+02:00
Minimum date for MSCI Minimum Volatility (MVOL.MI): 2015-08-01 00:00:00+02:00
Minimum date for Invesco Gold (SGLD.MI): 2015-08-01 00:00:00+02:00
Minimum date for GGOV Bond Eur (SXRQ.DE): 2015-08-01 00:00:00+02:00
Minimum date for Amundi Overnight return (LEONIA.MI): 2015-08-01 00:00:00+02:00


In [4]:
## suppose to allocate 50k eur with the percentages of NEFS on the 2022-04-01, and show the progressions of the portfolio until 2025-07-01
initial_investment = 1000
init_sum = 0
sec_sum = 0
for nef in NEFS:
    nef['initial_investment'] = (nef['percentage'] / 100) * initial_investment
    init_sum += nef['initial_investment']
    nef['progression'] = nef['data']['Close'].loc[min_date:].pct_change().fillna(0) * nef['initial_investment']
    nef['progression'] = nef['progression'].cumsum() + nef['initial_investment']
for etf in ETFS:
    etf['initial_investment'] = (etf['percentage'] / 100) * initial_investment
    sec_sum += etf['initial_investment']
    etf['progression'] = etf['data']['Close'].loc[min_date:].pct_change().fillna(0) * etf['initial_investment']
    etf['progression'] = etf['progression'].cumsum() + etf['initial_investment']
# for each nef, change the keys in the progression, to be the strftime(%Y-%m-%d) of the index
for nef in NEFS:
    nef['progression'] = nef['progression'].to_dict()
    nef['progression'] = {date.strftime('%Y-%m-%d'): value for date, value in nef['progression'].items()}
for etf in ETFS:
    etf['progression'] = etf['progression'].to_dict()
    etf['progression'] = {date.strftime('%Y-%m-%d'): value for date, value in etf['progression'].items()}
print(init_sum, sec_sum)

1000.0000000000001 1000.0


In [5]:
# across the progressions in NEFS, compute the total progression
for day in NEFS[0]['progression'].keys():
    monthly_sum = sum(nef['progression'][day] for nef in NEFS)
    etf_sum = sum(etf['progression'][day] for etf in ETFS)
    print(f"Total progression on {day}: {monthly_sum:.2f} EUR, ETFs: {etf_sum:.2f} EUR")

Total progression on 2022-04-01: 1000.00 EUR, ETFs: 1000.00 EUR
Total progression on 2022-05-01: 987.31 EUR, ETFs: 978.49 EUR
Total progression on 2022-06-01: 947.08 EUR, ETFs: 956.17 EUR
Total progression on 2022-07-01: 999.42 EUR, ETFs: 985.02 EUR
Total progression on 2022-08-01: 970.21 EUR, ETFs: 985.50 EUR
Total progression on 2022-09-01: 912.65 EUR, ETFs: 960.52 EUR
Total progression on 2022-10-01: 924.85 EUR, ETFs: 982.29 EUR
Total progression on 2022-11-01: 970.97 EUR, ETFs: 995.87 EUR
Total progression on 2022-12-01: 937.08 EUR, ETFs: 978.09 EUR
Total progression on 2023-01-01: 983.69 EUR, ETFs: 981.39 EUR
Total progression on 2023-02-01: 972.29 EUR, ETFs: 966.01 EUR
Total progression on 2023-03-01: 966.89 EUR, ETFs: 978.20 EUR
Total progression on 2023-04-01: 963.85 EUR, ETFs: 986.30 EUR
Total progression on 2023-05-01: 966.99 EUR, ETFs: 966.92 EUR
Total progression on 2023-06-01: 974.03 EUR, ETFs: 978.87 EUR
Total progression on 2023-07-01: 993.50 EUR, ETFs: 986.04 EUR
Total 

In [9]:
# for each fund, compute the return month by month
for nef in NEFS:
    nef['return'] = {date: (value - nef['initial_investment']) / nef['initial_investment'] * 100 for date, value in nef['progression'].items()}
for etf in ETFS:
    etf['return'] = {date: (value - etf['initial_investment']) / etf['initial_investment'] * 100 for date, value in etf['progression'].items()}

In [22]:
# using the returns, compute the standard deviation of the progression of each fund
import math
for nef in NEFS:
    prezzi_ordinati = nef['data']['Close'].loc[min_date:].pct_change().fillna(0)
    nef['std_dev'] = prezzi_ordinati.std()*100
for etf in ETFS:
    prezzi_ordinati = etf['data']['Close'].loc[min_date:].pct_change().fillna(0)
    etf['std_dev'] = prezzi_ordinati.std()*100
for nef in NEFS:
    print(f"{nef['name']} ({nef['ticker']}): {nef['std_dev']:.2f}%")
for etf in ETFS:
    print(f"{etf['name']} ({etf['ticker']}): {etf['std_dev']:.2f}%")

NEF Ethical Bond - Euro R Acc (0P00000I8O.F): 1.87%
NEF Ethical Global Equity R (0P00000I8C.F): 3.57%
NEF Ethical Euro Equity R Acc (0P00000I8A.F): 4.66%
NEF Emerging Market Equity R (0P00000I8E.F): 2.79%
NEF Emerging Market Bond R (0P00000I8S.F): 2.35%
NEF Ethical Balanced Conservative D (0P0001A36G.F): 1.93%
NEF Ethical Balanced Conservative R (0P0001A36E.F): 1.60%
NEF Ethical Balanced Dynamic D (0P00019L2P.F): 2.31%
NEF Ethical Balanced Dynamic R (0P00016B1E.F): 2.08%
NEF Ethical Corporate Bond - Euro R (0P00000I8I.F): 1.77%
NEF Ethical Global Trends R (0P0001IHXN.F): 4.25%
NEF Ethical Short Term bond - Euro R (0P00000I8G.F): 0.57%
NEF Ethical U.S. Equity R (0P00000GYL.F): 4.35%
NEF Global bond R (0P00000I8Q.F): 1.92%
NEF Global bond D (0P0001D4RR.F): 2.04%
NEF Pacific Equity R (0P00000GYH.F): 4.05%
NEF Risparmio Italia R (0P00000I8K.F): 1.86%
NEF Target 2028 D (0P0001MJ2E.F): 1.86%
NEF Schroder ISF Greater China A (0P0000G13X.F): 6.63%
MSCI World Momentum (IWFM.L): 3.68%
MSCI Minim

In [7]:
# compute the standard deviation of the progression of each fund
for nef in NEFS:
    nef['std_dev'] = np.std(list(nef['progression'].values()))
for etf in ETFS:
    etf['std_dev'] = np.std(list(etf['progression'].values()))
# print the standard deviation of each fund
for nef in NEFS:
    print(f"{nef['name']} ({nef['ticker']}): {nef['std_dev']:.2f} EUR")
for etf in ETFS:
    print(f"{etf['name']} ({etf['ticker']}): {etf['std_dev']:.2f} EUR")

NEF Ethical Bond - Euro R Acc (0P00000I8O.F): 1.71 EUR
NEF Ethical Global Equity R (0P00000I8C.F): 11.45 EUR
NEF Ethical Euro Equity R Acc (0P00000I8A.F): 6.79 EUR
NEF Emerging Market Equity R (0P00000I8E.F): 1.62 EUR
NEF Emerging Market Bond R (0P00000I8S.F): 2.09 EUR
NEF Ethical Balanced Conservative D (0P0001A36G.F): 0.83 EUR
NEF Ethical Balanced Conservative R (0P0001A36E.F): 0.91 EUR
NEF Ethical Balanced Dynamic D (0P00019L2P.F): 0.64 EUR
NEF Ethical Balanced Dynamic R (0P00016B1E.F): 4.21 EUR
NEF Ethical Corporate Bond - Euro R (0P00000I8I.F): 2.77 EUR
NEF Ethical Global Trends R (0P0001IHXN.F): 14.35 EUR
NEF Ethical Short Term bond - Euro R (0P00000I8G.F): 0.72 EUR
NEF Ethical U.S. Equity R (0P00000GYL.F): 10.38 EUR
NEF Global bond R (0P00000I8Q.F): 2.16 EUR
NEF Global bond D (0P0001D4RR.F): 0.51 EUR
NEF Pacific Equity R (0P00000GYH.F): 3.08 EUR
NEF Risparmio Italia R (0P00000I8K.F): 1.42 EUR
NEF Target 2028 D (0P0001MJ2E.F): 0.58 EUR
NEF Schroder ISF Greater China A (0P0000G13X